In [12]:
from validate_with_real import *
%load_ext autoreload
%autoreload 2


In [23]:
c=load_ground_truth(model_path="/Users/sarahurbut/Dropbox/resultshighamp/results/output_0_10000/model.pt")


/Users/sarahurbut/aladynoulli2/pyScripts/validate_with_real.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path)


In [31]:

Y,G,E,di=create_subset_data(c, n_individuals=1000,disease_indices=[66,112,127,17,25,55,198])


In [ ]:

def create_reference_trajectories(Y, initial_clusters, K, healthy_prop=0.01, frac=0.3):
    """
    Create reference trajectories using LOWESS smoothing with consistent normalization
    """
    from statsmodels.nonparametric.smoothers_lowess import lowess
    T = Y.shape[2]
    
    # Get raw counts
    Y_counts = Y.sum(dim=0)  # D x T
    signature_props = torch.zeros(K, T)
    
    # Get raw proportions WITHOUT cluster size normalization
    total_counts = Y_counts.sum(dim=0) + 1e-8
    for k in range(K):
        cluster_mask = (initial_clusters == k)
        signature_props[k] = Y_counts[cluster_mask].sum(dim=0) / Y_counts.sum(dim=0)
    
    # Normalize to sum to 1-healthy_prop at each timepoint
    signature_props = torch.clamp(signature_props, min=1e-8)
    signature_props = signature_props / signature_props.sum(dim=0, keepdim=True)
    signature_props *= (1 - healthy_prop)
    
    # Smooth using LOWESS
    times = np.arange(T)
    signature_refs = torch.zeros_like(signature_props)
    
    for k in range(K):
        smoothed = lowess(
            signature_props[k].numpy(), 
            times,
            frac=frac,
            it=3,
            delta=0.0,
            return_sorted=False
        )
        signature_refs[k] = torch.tensor(smoothed)
    
    # Re-normalize after smoothing to ensure proportions still sum correctly
    signature_refs = torch.clamp(signature_refs, min=1e-8)
    signature_refs = signature_refs / signature_refs.sum(dim=0, keepdim=True) * (1 - healthy_prop)
    
    # Convert to log space
    signature_refs = torch.log(signature_refs)
    healthy_ref = torch.ones(T) * torch.log(torch.tensor(healthy_prop, dtype=torch.float32))
    
    return signature_refs, healthy_ref

'Y'

In [3]:
from signature_recovery_sim import *

ImportError: cannot import name 'ClustHugeAmp' from 'clust_huge_amp' (/Users/sarahurbut/aladynoulli2/pyScripts/clust_huge_amp.py)